In [26]:
import pandas as pd
import numpy as np
import requests
import warnings
import openpyxl

In [3]:
# settings for cleaner output
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [6]:
# defining constants
api_key = "d17a24a4-3ec0-4ab2-8e40-73ddb37c415a"

In [7]:
# Important functions
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

In [18]:
# get rental property listings
# define the search url
rental_listing_url = "https://www.zillow.com/los-angeles-ca/rentals/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22mapBounds%22%3A%7B%22north%22%3A34.494381092115134%2C%22east%22%3A-117.60835725585937%2C%22south%22%3A33.545105377352435%2C%22west%22%3A-119.21510774414062%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%7D"

# get listings
rental_listings = get_listings(api_key, rental_listing_url)

In [9]:
# view all keys
rental_listings.json().keys()

dict_keys(['is_success', 'data', 'message'])

In [10]:
# check if request is successful
rental_listings.json()["is_success"]

True

In [11]:
# view count of properties returned in request
num_of_properties = rental_listings.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 9515


In [12]:
# view all listings
rental_listings_df = pd.json_normalize(rental_listings.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(rental_listings_df))
print("Number of columns:", len(rental_listings_df.columns))
rental_listings_df

Number of rows: 500
Number of columns: 82


,buildingId,lotId,price,minBeds,minBaths,minArea,imgSrc,hasImage,plid,isFeaturedListing,isShowcaseListing,unitCount,isHomeRec,buildingName,isBuilding,address,statusType,statusText,listingType,isFavorite,detailUrl,has3DModel,hasAdditionalAttributions,canSaveBuilding,timeOnZillow,latLong.latitude,latLong.longitude,variableData.type,variableData.text,badgeInfo.type,badgeInfo.text,badgeInfo,zpid,priceLabel,beds,baths,area,isUserClaimingOwner,isUserConfirmedClaim,visited,shouldShowZestimateAsPrice,pgapt,sgapt,hasVideo,availabilityDate,variableData.data.isFresh,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.isRentalWithBasePrice,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.unit,hdpData.homeInfo.isShowcaseListing,hdpData.homeInfo.livingArea,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceChange,hdpData.homeInfo.priceReduction,hdpData.homeInfo.zestimate,hdpData.homeInfo.taxAssessedValue,streetViewMetadataURL,streetViewURL
0,34.059753--118.467384,1.012242e+09,"$14,900+/mo",2.0,2.0,1554.0,https://photos.zillowstatic.com/fp/677baa48022...,True,4vdx0unm4gwbh,True,False,2.0,False,Respara,True,"11600 Dunstan Way, Los Angeles, CA",FOR_RENT,Respara,,False,/apartments/los-angeles-ca/respara/5ZZvt9/,True,False,True,1572707539,34.059753,-118.467384,3D_HOME,3D Tour,SPECIAL_OFFER,Special Offer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34.047615--118.4614,2.077264e+09,"$3,450+/mo",0.0,1.0,555.0,https://photos.zillowstatic.com/fp/81c61e1cc3c...,True,3k0q09sr17xwz,True,False,11.0,False,The Landmark Los Angeles,True,"11750 Wilshire Blvd, Los Angeles, CA",FOR_RENT,The Landmark Los Angeles,,False,/apartments/los-angeles-ca/the-landmark-los-an...,True,False,True,233072982,34.047615,-118.461400,3D_HOME,3D Tour,SPECIAL_OFFER,Special Offer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34.052998--118.43673,1.055624e+09,"$13,500+/mo",4.0,4.5,2403.0,https://photos.zillowstatic.com/fp/e6f7e086002...,True,3eeyay6csr0t4,True,False,2.0,False,Lumiere at Malcolm - Luxury Homes,True,"1644 Malcolm Ave, Los Angeles, CA",FOR_RENT,Lumiere at Malcolm - Luxury Homes,,False,/apartments/los-angeles-ca/lumiere-at-malcolm-...,True,False,True,108086678,34.052998,-118.436730,3D_HOME,3D Tour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34.03962--118.55603,2.077636e+09,"$13,995+/mo",3.0,3.0,2929.0,https://photos.zillowstatic.com/fp/7b4dd5435fd...,True,56k68dtqrg9c,True,False,7.0,False,Sea View Villas,True,"17325 Castellammare Dr, Pacific Palisades, CA",FOR_RENT,Sea View Villas,,False,/b/sea-view-villas-pacific-palisades-ca-9MPbzq/,True,False,True,847616822,34.039620,-118.556030,3D_HOME,3D Tour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34.06364--118.41437,1.008637e+09,"$10,400+/

In [17]:
rental_listings_df.columns

Index(['buildingId', 'lotId', 'price', 'minBeds', 'minBaths', 'minArea',
       'imgSrc', 'hasImage', 'plid', 'isFeaturedListing', 'isShowcaseListing',
       'unitCount', 'isHomeRec', 'buildingName', 'isBuilding', 'address',
       'statusType', 'statusText', 'listingType', 'isFavorite', 'detailUrl',
       'has3DModel', 'hasAdditionalAttributions', 'canSaveBuilding',
       'timeOnZillow', 'latLong.latitude', 'latLong.longitude',
       'variableData.type', 'variableData.text', 'badgeInfo.type',
       'badgeInfo.text', 'badgeInfo', 'zpid', 'priceLabel', 'beds', 'baths',
       'area', 'isUserClaimingOwner', 'isUserConfirmedClaim', 'visited',
       'shouldShowZestimateAsPrice', 'pgapt', 'sgapt', 'hasVideo',
       'availabilityDate', 'variableData.data.isFresh',
       'hdpData.homeInfo.zpid', 'hdpData.homeInfo.streetAddress',
       'hdpData.homeInfo.zipcode', 'hdpData.homeInfo.city',
       'hdpData.homeInfo.state', 'hdpData.homeInfo.latitude',
       'hdpData.homeInfo.longitude',

In [20]:
# get property for sale listings
# define the search url
for_sale_listing_url = "https://www.zillow.com/los-angeles-ca/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22mapBounds%22%3A%7B%22north%22%3A34.494381092115134%2C%22east%22%3A-117.60835725585937%2C%22south%22%3A33.545105377352435%2C%22west%22%3A-119.21510774414062%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%20CA%22%7D"

# get listings
for_sale_listings = get_listings(api_key, for_sale_listing_url)

In [21]:
# view all keys
for_sale_listings.json().keys()

dict_keys(['is_success', 'data', 'message'])

In [22]:
# check if request is successful
for_sale_listings.json()["is_success"]

True

In [23]:
# view count of properties returned in request
num_of_properties = for_sale_listings.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 5194


In [24]:
# view all listings
for_sale_listings_df = pd.json_normalize(for_sale_listings.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(for_sale_listings_df))
print("Number of columns:", len(for_sale_listings_df.columns))
for_sale_listings_df

Number of rows: 500
Number of columns: 92


,zpid,price,priceLabel,beds,baths,area,statusType,statusText,isFavorite,isUserClaimingOwner,isUserConfirmedClaim,imgSrc,hasImage,visited,listingType,shouldShowZestimateAsPrice,detailUrl,pgapt,sgapt,has3DModel,hasVideo,isHomeRec,address,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,availabilityDate,timeOnZillow,latLong.latitude,latLong.longitude,variableData.type,variableData.text,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,variableData,hdpData.homeInfo.unit,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceChange,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.open_house_info.open_house_showing,variableData.data.isFresh,hdpData.homeInfo.priceReduction,buildingId,lotId,minBeds,minBaths,minArea,unitCount,isBuilding,badgeInfo,canSaveBuilding,hdpData.homeInfo.videoCount,streetViewMetadataURL,streetViewURL,hdpData.homeInfo.listing_sub_type.is_newHome,hdpData.homeInfo.newConstructionType,lotAreaString,hdpData.homeInfo.listing_sub_type.is_bankOwned,hdpData.homeInfo.listing_sub_type.is_foreclosure,hdpData.homeInfo.isRentalWithBasePrice,hdpData.homeInfo.listing_sub_type.is_forAuction
0,20106772,"$899,000",$899K,5.0,3.0,2098.0,FOR_SALE,House for sale,False,False,False,https://photos.zillowstatic.com/fp/bdb073c29f2...,True,False,,False,/homedetails/15507-La-Valle-St-Sylmar-CA-91342...,ForSale,For Sale (Broker),True,False,False,"15507 La Valle St, Sylmar, CA 91342",False,False,False,None,321482355,34.299423,-118.471350,3D_HOME,3D Tour,20106772.0,15507 La Valle St,91342,Sylmar,CA,34.299423,-118.471350,899000.0,3.0,5.0,2098.0,SINGLE_FAMILY,FOR_SALE,-1.0,False,False,934100.0,4999.0,True,False,False,FOR_SALE,899000.0,True,False,False,USD,USA,870000.0,7742.000000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20621353,"$399,900",$400K,4.0,2.0,1922.0,FOR_SALE,House for sale,False,False,False,https://photos.zillowstatic.com/fp/e559f1fafac...,True,False,,False,/homedetails/1144-E-Adams-Blvd-Los-Angeles-CA-...,ForSale,For Sale (Broker),False,False,False,"1144 E Adams Blvd, Los Angeles, CA 90011",False,False,False,None,237246852,34.017876,-118.253630,NaN,NaN,20621353.0,1144 E Adams Blvd,90011,Los Angeles,CA,34.017876,-118.253630,399900.0,2.0,4.0,1922.0,SINGLE_FAMILY,FOR_SALE,-1.0,False,False,406400.0,3465.0,True,False,False,FOR_SALE,399900.0,True,False,False,USD,USA,25112.0,6359.760000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20506888,"$1,225,000",$1.23M,3.0,3.0,1632.0,FOR_SALE,Townhouse for sale,False,False,False,https://photos.zillowstatic.com/fp/d709172799b...,True,False,,False,/homedetails/10600-Holman-Ave-APT-5-Los-Angele...,ForSale,For Sale (Broker),True,False,False,"10600 Holman Ave APT 5, Los Angeles, CA 90024",False,False,False,None,321482355,34.054970,-118.430850,3D_HOME,3D Tour,20506888.0,10600 Holman Ave APT 5,90024,Los Angeles,CA,34.054970,-118.430850,1225000.0,3.0,3.0,1632.0,TOWNHOUSE,FOR_SALE,-1.0,False,False,1287500.0,4994.0,True,Fa

In [27]:
with pd.ExcelWriter('listings.xlsx') as writer:  
    rental_listings_df.to_excel(writer, sheet_name='rental')
    for_sale_listings_df.to_excel(writer, sheet_name='for_sale')

In [28]:
# get property for sale listings
# define the search url
agents_url = "https://www.zillow.com/professionals/real-estate-agent-reviews/los-angeles-ca/"

# get listings
agents = get_listings(api_key, agents_url)

In [29]:
# view all keys
agents.json().keys()

dict_keys(['is_success', 'data', 'message'])

In [30]:
# view count of properties returned in request
num_of_properties = agents.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

KeyError: 'categoryTotals'